# Naive Shapeshifting Algorithm

In this Notebook we will implement a naive solution for solving the shapeshifting problem. We do this by using one coordinator as the root of the new implementation and the other coordinator boat to move workers to different locations on the new configuration

## Methodology

To solve this problem we will take advantage of the geometries of the boats and how they are connected to find
the maximum overlap between our current configuration and our goal configuration at each step.


In [39]:
%load_ext autoreload
%autoreload 2

import graph_tool.all as gt
from naiveshapeshift import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Examples


## Identity
![title](img/shapeshifttest0.png)

In [159]:
# Make initial configuration
init_config = gt.Graph(directed = True)
coordinator = init_config.add_vertex()
side_boat = init_config.add_vertex()
latch = init_config.add_edge(coordinator, side_boat)
latch2 = init_config.add_edge(side_boat, coordinator)
edge_types = init_config.new_edge_property("int")
edge_types[latch] = 1
edge_types[latch2] = 3

In [160]:
# Make goal configuration, it's the same
goal_config = gt.Graph(directed = True)
goal_coord = goal_config.add_vertex()
goal_side = goal_config.add_vertex()
goal_latch = goal_config.add_edge(goal_coord, goal_side)
goal_latch2 = goal_config.add_edge(goal_side, goal_coord)
goal_edge_types = goal_config.new_edge_property("int")
goal_edge_types[goal_latch] = 1
goal_edge_types[goal_latch2] = 3

In [161]:
latching_plan = shapeshift(init_config, goal_config)
print(latching_plan)

[[(<Vertex object with index '1' at 0x13030d630>, <Vertex object with index '1' at 0x13030db10>)]]


## Thesis proposal
![title](img/shapeshifttest1.png)

In [171]:
init_config = gt.Graph(directed = True)
# initialize boats
init_coord = init_config.add_vertex()
init_work1 = init_config.add_vertex()
init_work2 = init_config.add_vertex()
init_work3 = init_config.add_vertex()
init_work4 = init_config.add_vertex()
init_work5 = init_config.add_vertex()

# initialize edges
i_coord_work1 = init_config.add_edge(init_coord, init_work1)
i_work1_coord = init_config.add_edge(init_work1, init_coord)

i_coord_work2 = init_config.add_edge(init_coord, init_work2)
i_work2_coord = init_config.add_edge(init_work2, init_coord)

i_work1_work3 = init_config.add_edge(init_work1, init_work3)
i_work3_work1 = init_config.add_edge(init_work3, init_work1)

i_work2_work3 = init_config.add_edge(init_work2, init_work3)
i_work3_work2 = init_config.add_edge(init_work3, init_work2)

i_work2_work4 = init_config.add_edge(init_work2, init_work4)
i_work4_work2 = init_config.add_edge(init_work4, init_work2)

i_work3_work5 = init_config.add_edge(init_work3, init_work5)
i_work5_work3 = init_config.add_edge(init_work5, init_work3)

i_work4_work5 = init_config.add_edge(init_work4, init_work5)
i_work5_work4 = init_config.add_edge(init_work5, init_work4)

# label edges
init_edge_types = init_config.new_edge_property("int")
init_edge_types[i_coord_work1] = 1
init_edge_types[i_work1_coord] = 3

init_edge_types[i_coord_work2] = 2
init_edge_types[i_work2_coord] = 4

init_edge_types[i_work1_work3] = 2
init_edge_types[i_work3_work1] = 4

init_edge_types[i_work2_work3] = 1
init_edge_types[i_work3_work2] = 3

init_edge_types[i_work2_work4] = 2
init_edge_types[i_work4_work2] = 4

init_edge_types[i_work3_work5] = 2
init_edge_types[i_work5_work3] = 4

init_edge_types[i_work4_work5] = 1
init_edge_types[i_work5_work4] = 3
init_config.edge_properties['connection_type'] = init_edge_types

In [172]:
goal_config = gt.Graph(directed = True)
#initialize boats
goal_coord = goal_config.add_vertex()
goal_work1 = goal_config.add_vertex()
goal_work2 = goal_config.add_vertex()
goal_work3 = goal_config.add_vertex()
goal_work4 = goal_config.add_vertex()
goal_work5 = goal_config.add_vertex()

#initialize edges
g_work1_work2 = goal_config.add_edge(goal_work1, goal_work2)
g_work2_work1 = goal_config.add_edge(goal_work2, goal_work1)

g_work1_work4 = goal_config.add_edge(goal_work1, goal_work4)
g_work4_work1 = goal_config.add_edge(goal_work4, goal_work1)

g_work2_work3 = goal_config.add_edge(goal_work2, goal_work3)
g_work3_work2 = goal_config.add_edge(goal_work3, goal_work2)

g_work2_work5 = goal_config.add_edge(goal_work2, goal_work5)
g_work5_work2 = goal_config.add_edge(goal_work5, goal_work2)

g_work3_coord = goal_config.add_edge(goal_work3, goal_coord)
g_coord_work3 = goal_config.add_edge(goal_coord, goal_work3)

g_work4_work5 = goal_config.add_edge(goal_work4, goal_work5)
g_work5_work4 = goal_config.add_edge(goal_work5, goal_work4)

g_work5_coord = goal_config.add_edge(goal_work5, goal_coord)
g_coord_work5 = goal_config.add_edge(goal_coord, goal_work5)

#label edges
goal_edge_types = goal_config.new_edge_property("int")
goal_edge_types[g_work1_work2] = 1
goal_edge_types[g_work2_work1] = 3

goal_edge_types[g_work1_work4] = 2
goal_edge_types[g_work4_work1] = 4

goal_edge_types[g_work2_work3] = 1
goal_edge_types[g_work3_work2] = 3

goal_edge_types[g_work2_work5] = 2
goal_edge_types[g_work5_work2] = 4

goal_edge_types[g_work3_coord] = 2
goal_edge_types[g_coord_work3] = 4

goal_edge_types[g_work4_work5] = 1
goal_edge_types[g_work5_work4] = 3

goal_edge_types[g_work5_coord] = 1
goal_edge_types[g_coord_work5] = 3
goal_config.edge_properties['connection_type'] = goal_edge_types

In [173]:
latching_plan = shapeshift(init_config, goal_config)
print(latching_plan)

[[(<Vertex object with index '5' at 0x13030d2d0>, <Vertex object with index '5' at 0x13015f630>), (<Vertex object with index '4' at 0x13015f5d0>, <Vertex object with index '4' at 0x13015f7b0>), (<Vertex object with index '2' at 0x13015ff90>, <Vertex object with index '1' at 0x10e8d2870>), (<Vertex object with index '3' at 0x13015f990>, <Vertex object with index '2' at 0x13015fe10>)], [(<Vertex object with index '1' at 0x13030d930>, <Vertex object with index '3' at 0x13030dcf0>)]]


# Reverse Thesis proposal
![title](img/shapeshifttest2.png)

In [174]:
init_config = gt.Graph(directed = True)
# initialize boats
init_coord = init_config.add_vertex()
init_work1 = init_config.add_vertex()
init_work2 = init_config.add_vertex()
init_work3 = init_config.add_vertex()
init_work4 = init_config.add_vertex()
init_work5 = init_config.add_vertex()

# initialize edges
i_coord_work1 = init_config.add_edge(init_coord, init_work1)
i_work1_coord = init_config.add_edge(init_work1, init_coord)

i_coord_work2 = init_config.add_edge(init_coord, init_work2)
i_work2_coord = init_config.add_edge(init_work2, init_coord)

i_work1_work3 = init_config.add_edge(init_work1, init_work3)
i_work3_work1 = init_config.add_edge(init_work3, init_work1)

i_work2_work3 = init_config.add_edge(init_work2, init_work3)
i_work3_work2 = init_config.add_edge(init_work3, init_work2)

i_work2_work4 = init_config.add_edge(init_work2, init_work4)
i_work4_work2 = init_config.add_edge(init_work4, init_work2)

i_work3_work5 = init_config.add_edge(init_work3, init_work5)
i_work5_work3 = init_config.add_edge(init_work5, init_work3)

i_work4_work5 = init_config.add_edge(init_work4, init_work5)
i_work5_work4 = init_config.add_edge(init_work5, init_work4)

# label edges
init_edge_types = init_config.new_edge_property("int")
init_edge_types[i_coord_work1] = 1
init_edge_types[i_work1_coord] = 3

init_edge_types[i_coord_work2] = 2
init_edge_types[i_work2_coord] = 4

init_edge_types[i_work1_work3] = 2
init_edge_types[i_work3_work1] = 4

init_edge_types[i_work2_work3] = 1
init_edge_types[i_work3_work2] = 3

init_edge_types[i_work2_work4] = 2
init_edge_types[i_work4_work2] = 4

init_edge_types[i_work3_work5] = 2
init_edge_types[i_work5_work3] = 4

init_edge_types[i_work4_work5] = 1
init_edge_types[i_work5_work4] = 3
init_config.edge_properties['connection_type'] = init_edge_types

In [175]:
goal_config = gt.Graph(directed = True)
#initialize boats
goal_coord = goal_config.add_vertex()
goal_work1 = goal_config.add_vertex()
goal_work2 = goal_config.add_vertex()
goal_work3 = goal_config.add_vertex()
goal_work4 = goal_config.add_vertex()
goal_work5 = goal_config.add_vertex()

#initialize edges
g_work1_work2 = goal_config.add_edge(goal_work1, goal_work2)
g_work2_work1 = goal_config.add_edge(goal_work2, goal_work1)

g_work1_work4 = goal_config.add_edge(goal_work1, goal_work4)
g_work4_work1 = goal_config.add_edge(goal_work4, goal_work1)

g_work2_work3 = goal_config.add_edge(goal_work2, goal_work3)
g_work3_work2 = goal_config.add_edge(goal_work3, goal_work2)

g_work2_work5 = goal_config.add_edge(goal_work2, goal_work5)
g_work5_work2 = goal_config.add_edge(goal_work5, goal_work2)

g_work3_coord = goal_config.add_edge(goal_work3, goal_coord)
g_coord_work3 = goal_config.add_edge(goal_coord, goal_work3)

g_work4_work5 = goal_config.add_edge(goal_work4, goal_work5)
g_work5_work4 = goal_config.add_edge(goal_work5, goal_work4)

g_work5_coord = goal_config.add_edge(goal_work5, goal_coord)
g_coord_work5 = goal_config.add_edge(goal_coord, goal_work5)

#label edges
goal_edge_types = goal_config.new_edge_property("int")
goal_edge_types[g_work1_work2] = 1
goal_edge_types[g_work2_work1] = 3

goal_edge_types[g_work1_work4] = 2
goal_edge_types[g_work4_work1] = 4

goal_edge_types[g_work2_work3] = 1
goal_edge_types[g_work3_work2] = 3

goal_edge_types[g_work2_work5] = 2
goal_edge_types[g_work5_work2] = 4

goal_edge_types[g_work3_coord] = 2
goal_edge_types[g_coord_work3] = 4

goal_edge_types[g_work4_work5] = 1
goal_edge_types[g_work5_work4] = 3

goal_edge_types[g_work5_coord] = 1
goal_edge_types[g_coord_work5] = 3
goal_config.edge_properties['connection_type'] = goal_edge_types

In [176]:
latching_plan = shapeshift(goal_config, init_config)
print(latching_plan)

[[(<Vertex object with index '1' at 0x13030df30>, <Vertex object with index '2' at 0x13015fe70>), (<Vertex object with index '4' at 0x13015fc30>, <Vertex object with index '4' at 0x13015fbd0>), (<Vertex object with index '5' at 0x13015f6f0>, <Vertex object with index '5' at 0x13015fab0>), (<Vertex object with index '2' at 0x13030d690>, <Vertex object with index '3' at 0x13030d450>)], [(<Vertex object with index '3' at 0x13015f810>, <Vertex object with index '1' at 0x13015ff30>)]]


# Last boat no coordinator connection
![title](img/shapeshifttest3.png)

In [12]:
init_config = gt.Graph(directed = True)
# initialize boats
init_coord = init_config.add_vertex()
init_work1 = init_config.add_vertex()
init_work2 = init_config.add_vertex()
init_work3 = init_config.add_vertex()
init_work4 = init_config.add_vertex()
init_work5 = init_config.add_vertex()

# initialize edges
i_coord_work1 = init_config.add_edge(init_coord, init_work1)
i_work1_coord = init_config.add_edge(init_work1, init_coord)

i_coord_work2 = init_config.add_edge(init_coord, init_work2)
i_work2_coord = init_config.add_edge(init_work2, init_coord)

i_work1_work3 = init_config.add_edge(init_work1, init_work3)
i_work3_work1 = init_config.add_edge(init_work3, init_work1)

i_work2_work3 = init_config.add_edge(init_work2, init_work3)
i_work3_work2 = init_config.add_edge(init_work3, init_work2)

i_work2_work4 = init_config.add_edge(init_work2, init_work4)
i_work4_work2 = init_config.add_edge(init_work4, init_work2)

i_work3_work5 = init_config.add_edge(init_work3, init_work5)
i_work5_work3 = init_config.add_edge(init_work5, init_work3)

i_work4_work5 = init_config.add_edge(init_work4, init_work5)
i_work5_work4 = init_config.add_edge(init_work5, init_work4)

# label edges
init_edge_types = init_config.new_edge_property("int")
init_edge_types[i_coord_work1] = 1
init_edge_types[i_work1_coord] = 3

init_edge_types[i_coord_work2] = 2
init_edge_types[i_work2_coord] = 4

init_edge_types[i_work1_work3] = 2
init_edge_types[i_work3_work1] = 4

init_edge_types[i_work2_work3] = 1
init_edge_types[i_work3_work2] = 3

init_edge_types[i_work2_work4] = 2
init_edge_types[i_work4_work2] = 4

init_edge_types[i_work3_work5] = 2
init_edge_types[i_work5_work3] = 4

init_edge_types[i_work4_work5] = 1
init_edge_types[i_work5_work4] = 3
init_config.edge_properties['connection_type'] = init_edge_types

In [13]:
goal_config = gt.Graph(directed = True)
#initialize boats
goal_coord = goal_config.add_vertex()
goal_work1 = goal_config.add_vertex()
goal_work2 = goal_config.add_vertex()
goal_work3 = goal_config.add_vertex()
goal_work4 = goal_config.add_vertex()
goal_work5 = goal_config.add_vertex()

#initialize edges
g_work1_work2 = goal_config.add_edge(goal_work1, goal_work2)
g_work2_work1 = goal_config.add_edge(goal_work2, goal_work1)

g_work1_work4 = goal_config.add_edge(goal_work1, goal_work4)
g_work4_work1 = goal_config.add_edge(goal_work4, goal_work1)

g_work3_work4 = goal_config.add_edge(goal_work3, goal_work4)
g_work4_work3 = goal_config.add_edge(goal_work4, goal_work3)

g_work2_work5 = goal_config.add_edge(goal_work2, goal_work5)
g_work5_work2 = goal_config.add_edge(goal_work5, goal_work2)

g_work4_work5 = goal_config.add_edge(goal_work4, goal_work5)
g_work5_work4 = goal_config.add_edge(goal_work5, goal_work4)

g_work5_coord = goal_config.add_edge(goal_work5, goal_coord)
g_coord_work5 = goal_config.add_edge(goal_coord, goal_work5)

#label edges
goal_edge_types = goal_config.new_edge_property("int")
goal_edge_types[g_work1_work2] = 1
goal_edge_types[g_work2_work1] = 3

goal_edge_types[g_work1_work4] = 2
goal_edge_types[g_work4_work1] = 4

goal_edge_types[g_work3_work4] = 1
goal_edge_types[g_work4_work3] = 3

goal_edge_types[g_work2_work5] = 2
goal_edge_types[g_work5_work2] = 4

goal_edge_types[g_work4_work5] = 1
goal_edge_types[g_work5_work4] = 3

goal_edge_types[g_work5_coord] = 1
goal_edge_types[g_coord_work5] = 3
goal_config.edge_properties['connection_type'] = goal_edge_types

In [14]:
latching_plan = shapeshift(init_config, goal_config)
print(latching_plan)

[[(<Vertex object with index '5' at 0x130878690>, <Vertex object with index '5' at 0x130878750>), (<Vertex object with index '4' at 0x130878810>, <Vertex object with index '4' at 0x130878870>), (<Vertex object with index '2' at 0x130878990>, <Vertex object with index '1' at 0x1308789f0>), (<Vertex object with index '3' at 0x130878450>, <Vertex object with index '2' at 0x1308783f0>)], [(<Vertex object with index '1' at 0x1308788d0>, <Vertex object with index '3' at 0x130878ab0>)]]


# Geometric Constraint
![title](img/shapeshifttest4.png)

In [177]:
init_config = gt.Graph(directed = True)
# initialize boats
init_coord = init_config.add_vertex()
init_work1 = init_config.add_vertex()
init_work2 = init_config.add_vertex()
init_work3 = init_config.add_vertex()
init_work4 = init_config.add_vertex()
init_work5 = init_config.add_vertex()
init_work6 = init_config.add_vertex()

#initialize edges
i_coord_work1 = init_config.add_edge(init_coord, init_work1)
i_work1_coord = init_config.add_edge(init_work1, init_coord)

i_work1_work2 = init_config.add_edge(init_work1, init_work2)
i_work2_work1 = init_config.add_edge(init_work2, init_work1)

i_work2_work4 = init_config.add_edge(init_work2, init_work4)
i_work4_work2 = init_config.add_edge(init_work4, init_work2)

i_work3_work6 = init_config.add_edge(init_work3, init_work6)
i_work6_work3 = init_config.add_edge(init_work6, init_work3)

i_work4_work5 = init_config.add_edge(init_work4, init_work5)
i_work5_work4 = init_config.add_edge(init_work5, init_work4)

i_work5_work6 = init_config.add_edge(init_work5, init_work6)
i_work6_work5 = init_config.add_edge(init_work6, init_work5)

#label edges
init_edge_types = init_config.new_edge_property("int")
init_edge_types[i_coord_work1] = 2
init_edge_types[i_work1_coord] = 4

init_edge_types[i_work1_work2] = 1
init_edge_types[i_work2_work1] = 3

init_edge_types[i_work2_work4] = 2
init_edge_types[i_work4_work2] = 4

init_edge_types[i_work4_work5] = 1
init_edge_types[i_work5_work4] = 3

init_edge_types[i_work5_work6] = 1
init_edge_types[i_work6_work5] = 3

init_edge_types[i_work3_work6] = 2
init_edge_types[i_work6_work3] = 4
init_config.edge_properties['connection_type'] = init_edge_types

In [178]:
goal_config = gt.Graph(directed = True)
#initialize boats
goal_coord = goal_config.add_vertex()
goal_work1 = goal_config.add_vertex()
goal_work2 = goal_config.add_vertex()
goal_work3 = goal_config.add_vertex()
goal_work4 = goal_config.add_vertex()
goal_work5 = goal_config.add_vertex()
goal_work6 = goal_config.add_vertex()

#initialize edges
g_work1_work2 = goal_config.add_edge(goal_work1, goal_work2)
g_work2_work1 = goal_config.add_edge(goal_work2, goal_work1)

g_work1_work4 = goal_config.add_edge(goal_work1, goal_work4)
g_work4_work1 = goal_config.add_edge(goal_work4, goal_work1)

g_work2_work3 = goal_config.add_edge(goal_work2, goal_work3)
g_work3_work2 = goal_config.add_edge(goal_work3, goal_work2)

g_work2_work5 = goal_config.add_edge(goal_work2, goal_work5)
g_work5_work2 = goal_config.add_edge(goal_work5, goal_work2)

g_work4_work5 = goal_config.add_edge(goal_work4, goal_work5)
g_work5_work4 = goal_config.add_edge(goal_work5, goal_work4)

g_work5_work6 = goal_config.add_edge(goal_work5, goal_work6)
g_work6_work5 = goal_config.add_edge(goal_work6, goal_work5)

g_work3_work6 = goal_config.add_edge(goal_work3, goal_work6)
g_work6_work3 = goal_config.add_edge(goal_work6, goal_work3)

g_work6_coord = goal_config.add_edge(goal_work6, goal_coord)
g_coord_work6 = goal_config.add_edge(goal_coord, goal_work6)

#label edges
goal_edge_types = goal_config.new_edge_property("int")
goal_edge_types[g_work1_work2] = 1
goal_edge_types[g_work2_work1] = 3

goal_edge_types[g_work1_work4] = 2
goal_edge_types[g_work4_work1] = 4

goal_edge_types[g_work2_work3] = 1
goal_edge_types[g_work3_work2] = 3

goal_edge_types[g_work2_work5] = 2
goal_edge_types[g_work5_work2] = 4

goal_edge_types[g_work4_work5] = 1
goal_edge_types[g_work5_work4] = 3

goal_edge_types[g_work5_work6] = 1
goal_edge_types[g_work6_work5] = 3

goal_edge_types[g_work3_work6] = 2
goal_edge_types[g_work6_work3] = 4

goal_edge_types[g_work6_coord] = 1
goal_edge_types[g_coord_work6] = 3
goal_config.edge_properties['connection_type'] = goal_edge_types

In [179]:
latching_plan = shapeshift(init_config, goal_config)
print(latching_plan)

[[(<Vertex object with index '3' at 0x13030df90>, <Vertex object with index '6' at 0x13030d7b0>)], [(<Vertex object with index '6' at 0x13030d630>, <Vertex object with index '3' at 0x13030d750>), (<Vertex object with index '5' at 0x13030dab0>, <Vertex object with index '2' at 0x13030dcf0>), (<Vertex object with index '4' at 0x13030db10>, <Vertex object with index '1' at 0x13030d210>)], [(<Vertex object with index '2' at 0x13030d9f0>, <Vertex object with index '5' at 0x13030d510>), (<Vertex object with index '1' at 0x13030dbd0>, <Vertex object with index '4' at 0x13030d3f0>)]]


# Geometric Constraint 2
![title](img/shapeshifttest5.png)

In [180]:
init_config = gt.Graph(directed = True)
# initialize boats
init_coord = init_config.add_vertex()
init_work1 = init_config.add_vertex()
init_work2 = init_config.add_vertex()
init_work3 = init_config.add_vertex()
init_work4 = init_config.add_vertex()
init_work5 = init_config.add_vertex()
init_work6 = init_config.add_vertex()

#initialize edges
i_coord_work1 = init_config.add_edge(init_coord, init_work1)
i_work1_coord = init_config.add_edge(init_work1, init_coord)

i_work1_work2 = init_config.add_edge(init_work1, init_work2)
i_work2_work1 = init_config.add_edge(init_work2, init_work1)

i_work2_work4 = init_config.add_edge(init_work2, init_work4)
i_work4_work2 = init_config.add_edge(init_work4, init_work2)

i_work3_work6 = init_config.add_edge(init_work3, init_work6)
i_work6_work3 = init_config.add_edge(init_work6, init_work3)

i_work4_work5 = init_config.add_edge(init_work4, init_work5)
i_work5_work4 = init_config.add_edge(init_work5, init_work4)

i_work5_work6 = init_config.add_edge(init_work5, init_work6)
i_work6_work5 = init_config.add_edge(init_work6, init_work5)

#label edges
init_edge_types = init_config.new_edge_property("int")
init_edge_types[i_coord_work1] = 2
init_edge_types[i_work1_coord] = 4

init_edge_types[i_work1_work2] = 1
init_edge_types[i_work2_work1] = 3

init_edge_types[i_work2_work4] = 2
init_edge_types[i_work4_work2] = 4

init_edge_types[i_work4_work5] = 1
init_edge_types[i_work5_work4] = 3

init_edge_types[i_work5_work6] = 1
init_edge_types[i_work6_work5] = 3

init_edge_types[i_work3_work6] = 2
init_edge_types[i_work6_work3] = 4
init_config.edge_properties['connection_type'] = init_edge_types

In [181]:
goal_config = gt.Graph(directed = True)
#initialize boats
goal_coord = goal_config.add_vertex()
goal_work1 = goal_config.add_vertex()
goal_work2 = goal_config.add_vertex()
goal_work3 = goal_config.add_vertex()
goal_work4 = goal_config.add_vertex()
goal_work5 = goal_config.add_vertex()
goal_work6 = goal_config.add_vertex()

#initialize edges
g_work1_work2 = goal_config.add_edge(goal_work1, goal_work2)
g_work2_work1 = goal_config.add_edge(goal_work2, goal_work1)

g_work1_work4 = goal_config.add_edge(goal_work1, goal_work4)
g_work4_work1 = goal_config.add_edge(goal_work4, goal_work1)

g_work2_work3 = goal_config.add_edge(goal_work2, goal_work3)
g_work3_work2 = goal_config.add_edge(goal_work3, goal_work2)

g_work2_work5 = goal_config.add_edge(goal_work2, goal_work5)
g_work5_work2 = goal_config.add_edge(goal_work5, goal_work2)

g_work4_work5 = goal_config.add_edge(goal_work4, goal_work5)
g_work5_work4 = goal_config.add_edge(goal_work5, goal_work4)

g_work5_work6 = goal_config.add_edge(goal_work5, goal_work6)
g_work6_work5 = goal_config.add_edge(goal_work6, goal_work5)

g_work3_work6 = goal_config.add_edge(goal_work3, goal_work6)
g_work6_work3 = goal_config.add_edge(goal_work6, goal_work3)

g_work3_coord = goal_config.add_edge(goal_work3, goal_coord)
g_coord_work3 = goal_config.add_edge(goal_coord, goal_work3)

#label edges
goal_edge_types = goal_config.new_edge_property("int")
goal_edge_types[g_work1_work2] = 1
goal_edge_types[g_work2_work1] = 3

goal_edge_types[g_work1_work4] = 2
goal_edge_types[g_work4_work1] = 4

goal_edge_types[g_work2_work3] = 1
goal_edge_types[g_work3_work2] = 3

goal_edge_types[g_work2_work5] = 2
goal_edge_types[g_work5_work2] = 4

goal_edge_types[g_work4_work5] = 1
goal_edge_types[g_work5_work4] = 3

goal_edge_types[g_work5_work6] = 1
goal_edge_types[g_work6_work5] = 3

goal_edge_types[g_work3_work6] = 2
goal_edge_types[g_work6_work3] = 4

goal_edge_types[g_work3_coord] = 1
goal_edge_types[g_coord_work3] = 3
goal_config.edge_properties['connection_type'] = goal_edge_types

In [182]:
latching_plan = shapeshift(init_config, goal_config)
print(latching_plan)

vert2
[[(<Vertex object with index '3' at 0x13030d090>, <Vertex object with index '3' at 0x13030d450>), (<Vertex object with index '6' at 0x13015fcf0>, <Vertex object with index '6' at 0x13015ff90>), (<Vertex object with index '5' at 0x13030d6f0>, <Vertex object with index '5' at 0x13030d690>), (<Vertex object with index '4' at 0x130edf390>, <Vertex object with index '4' at 0x130edf270>)], [(<Vertex object with index '2' at 0x13015fd50>, <Vertex object with index '2' at 0x13015fb70>), (<Vertex object with index '1' at 0x13015f870>, <Vertex object with index '1' at 0x13015f6f0>)]]
